---

# Chapter 21: Common Anti-Patterns

## Opening Context

Design patterns are proven solutions to recurring problems. However, not every recurring solution is good—some are traps that lead to unmaintainable, fragile, or inefficient code. These traps are known as **anti‑patterns**. They often arise from well‑intentioned but misguided attempts to solve a problem quickly, or from applying a pattern in the wrong context. Recognising anti‑patterns is as important as knowing design patterns; it helps you avoid common pitfalls and refactor towards cleaner designs.

This chapter explores four notorious anti‑patterns:

1. **Spaghetti Code and Ravioli Code** – Two extremes of unstructured code that are hard to follow and modify.
2. **The God Object and Golden Hammer** – A class that does too much, and a tool that is applied everywhere regardless of suitability.
3. **Premature Optimization** – The root of many performance‑related evils.
4. **Copy‑Paste Programming** – The silent killer of maintainability.

Understanding these anti‑patterns will sharpen your design instincts and help you build systems that remain clean over time.

---

## 21.1 Spaghetti Code and Ravioli Code

### Intent (of the Anti‑Pattern)
*Describe code that lacks structure, making it nearly impossible to understand, test, or maintain.*

### Spaghetti Code

Spaghetti code is characterised by a tangled, unstructured control flow. It often results from overuse of `goto` statements (in older languages), deeply nested conditionals, and long methods that do everything. The code “winds around” like a plate of spaghetti, making it impossible to trace execution paths.

#### Example: Spaghetti Code in a Shopping Cart

```typescript
// ❌ SPAGHETTI CODE: Everything in one method, deeply nested, unclear flow
function checkoutProcessor(request: any): any {
  let result;
  if (request.user) {
    if (request.user.isLoggedIn) {
      if (request.items && request.items.length > 0) {
        let total = 0;
        for (let i = 0; i < request.items.length; i++) {
          let item = request.items[i];
          if (item.price) {
            if (item.quantity) {
              total += item.price * item.quantity;
            } else {
              // assume quantity 1
              total += item.price;
            }
          } else {
            // log error
            console.log('Item missing price');
          }
        }
        // apply discount if user is premium
        if (request.user.premium) {
          total = total * 0.9;
        }
        // check inventory (pseudo)
        if (inventoryCheck(request.items)) {
          // create order
          let order = {
            id: generateId(),
            user: request.user.id,
            items: request.items,
            total: total,
            status: 'pending'
          };
          // save to database
          db.orders.save(order, (err) => {
            if (err) {
              result = { error: 'Database error' };
            } else {
              result = { orderId: order.id };
            }
          });
        } else {
          result = { error: 'Out of stock' };
        }
      } else {
        result = { error: 'No items' };
      }
    } else {
      result = { error: 'Not logged in' };
    }
  } else {
    result = { error: 'User missing' };
  }
  return result;
}
```

**Problems**:
- **Deep nesting** – Multiple levels of `if/else` make it hard to follow.
- **Mixed concerns** – Validation, business logic, persistence, and error handling are all intertwined.
- **Poor readability** – The flow is linear but cluttered; you need to read the whole function to understand any part.
- **Untestable** – Testing requires simulating many nested conditions.

### Ravioli Code

Ravioli code is the opposite extreme: the code is broken into too many tiny, loosely connected pieces. Each piece is simple, but the interactions are hard to follow because the logic is scattered across many small functions or classes. It’s like ravioli—each piece is neat, but the overall dish is disconnected.

#### Example: Ravioli Code

```typescript
// ❌ RAVIOLI CODE: Too many tiny functions, logic scattered
function processRequest(req: any): any {
  const user = extractUser(req);
  if (!isUserValid(user)) return error('Invalid user');
  const items = extractItems(req);
  if (!areItemsValid(items)) return error('Invalid items');
  const total = calculateTotal(items);
  const discountedTotal = applyDiscount(user, total);
  if (!(await checkInventory(items))) return error('Out of stock');
  const order = createOrder(user, items, discountedTotal);
  const savedOrder = await saveOrder(order);
  return formatResponse(savedOrder);
}

function extractUser(req: any) { /* ... */ }
function isUserValid(user: any) { /* ... */ }
function extractItems(req: any) { /* ... */ }
function areItemsValid(items: any) { /* ... */ }
function calculateTotal(items: any) { /* ... */ }
function applyDiscount(user: any, total: number) { /* ... */ }
function checkInventory(items: any) { /* ... */ }
function createOrder(user: any, items: any, total: number) { /* ... */ }
function saveOrder(order: any) { /* ... */ }
function formatResponse(order: any) { /* ... */ }
function error(msg: string) { /* ... */ }
```

**Problems**:
- **Scattered logic** – The main flow is clear, but to understand what `applyDiscount` does, you have to jump to that function.
- **Hidden dependencies** – Functions may implicitly depend on global state or other functions.
- **Over‑abstraction** – Functions that are only used once and are trivial add indirection without benefit.
- **Testing overhead** – You need to test many tiny functions, often with overlapping scenarios.

### The Balance: Structured Code

Good code lies in the middle: it has clear, cohesive modules with well‑defined responsibilities, but not so many that the overall flow is lost. Methods are long enough to tell a story but short enough to fit in your head.

**Refactored Example**:

```typescript
// ✅ STRUCTURED CODE: Clear separation but not over‑fragmented
class CheckoutService {
  constructor(
    private inventoryService: InventoryService,
    private orderRepository: OrderRepository
  ) {}

  async checkout(userId: string, items: CartItem[]): Promise<CheckoutResult> {
    this.validateInput(userId, items);
    const user = await this.getUser(userId);
    const total = this.calculateTotal(items);
    const finalTotal = this.applyUserDiscount(user, total);
    await this.reserveInventory(items);
    const order = await this.createOrder(userId, items, finalTotal);
    return { orderId: order.id };
  }

  private validateInput(userId: string, items: CartItem[]): void {
    if (!userId) throw new Error('User required');
    if (!items.length) throw new Error('Cart empty');
  }

  private async getUser(userId: string): Promise<User> { /* ... */ }

  private calculateTotal(items: CartItem[]): number { /* ... */ }

  private applyUserDiscount(user: User, total: number): number {
    return user.isPremium ? total * 0.9 : total;
  }

  private async reserveInventory(items: CartItem[]): Promise<void> {
    if (!await this.inventoryService.check(items)) {
      throw new Error('Out of stock');
    }
  }

  private async createOrder(userId: string, items: CartItem[], total: number): Promise<Order> {
    const order = new Order(userId, items, total);
    return this.orderRepository.save(order);
  }
}
```

**Explanation**:
- The main flow is clear and linear.
- Helper methods are cohesive and at an appropriate level of abstraction.
- Dependencies are injected, making testing easier.

---

## 21.2 The God Object and Golden Hammer

### God Object

A **God Object** (or God Class) is a class that knows too much or does too much. It becomes the central point of the system, with many responsibilities. It often accumulates over time as developers add functionality to an existing class instead of creating new ones.

#### Example: God Object in an E‑commerce System

```typescript
// ❌ GOD OBJECT: Handles everything
class OrderManager {
  private db: Database;

  constructor() {
    this.db = new Database(); // direct dependency
  }

  createOrder(userData: any, items: any[]) { /* ... */ }
  calculateTax(order: any) { /* ... */ }
  applyDiscount(order: any, code: string) { /* ... */ }
  sendConfirmationEmail(order: any) { /* ... */ }
  updateInventory(items: any[]) { /* ... */ }
  generateInvoice(order: any) { /* ... */ }
  trackShipment(orderId: string) { /* ... */ }
  handleReturns(orderId: string) { /* ... */ }
  generateSalesReport(start: Date, end: Date) { /* ... */ }
  // ... 50 more methods
}
```

**Problems**:
- **Low cohesion** – Methods are unrelated (e.g., sales report generation and email sending).
- **High coupling** – Every other part of the system depends on this one class.
- **Hard to test** – You need to instantiate the whole god object to test any single method.
- **Difficult to maintain** – Changes ripple through unrelated features.

**Solution**: Decompose the god object into smaller, focused classes following the **Single Responsibility Principle**.

```typescript
// ✅ DECOMPOSED: Separate concerns
class OrderCreator { /* ... */ }
class TaxCalculator { /* ... */ }
class DiscountApplier { /* ... */ }
class EmailService { /* ... */ }
class InventoryUpdater { /* ... */ }
class InvoiceGenerator { /* ... */ }
class ShipmentTracker { /* ... */ }
class ReturnHandler { /* ... */ }
class SalesReporter { /* ... */ }
```

### Golden Hammer

The **Golden Hammer** anti‑pattern occurs when a developer has a favourite tool, technique, or pattern and applies it to every problem, regardless of suitability. “If all you have is a hammer, everything looks like a nail.”

#### Example: Overusing Singleton

A developer loves the Singleton pattern and uses it for everything—configuration, logging, database connections, even for services that should be instantiated per request.

```typescript
// ❌ GOLDEN HAMMER: Singleton for everything
class ConfigManager {
  private static instance: ConfigManager;
  private constructor() {}
  static getInstance() { /* ... */ }
}

class Logger {
  private static instance: Logger;
  private constructor() {}
  static getInstance() { /* ... */ }
}

class UserService {
  private static instance: UserService;
  private constructor() {}
  static getInstance() { /* ... */ }
}

class OrderService {
  private static instance: OrderService;
  private constructor() {}
  static getInstance() { /* ... */ }
}
```

**Problems**:
- Singletons introduce global state, making unit testing difficult.
- They hide dependencies (a class that uses `UserService.getInstance()` has an invisible dependency).
- They can cause contention in multi‑threaded environments.

**Better**: Use dependency injection and appropriate scoping (e.g., per‑request for services, singleton only where truly needed, like configuration).

**Other Golden Hammer examples**:
- Using microservices for a simple CRUD app because they are trendy.
- Applying complex design patterns (e.g., Visitor) to a simple problem.
- Using a relational database for document‑oriented data because you know SQL well.

**Solution**: Step back, analyse the problem, and choose the simplest solution that works. Be aware of biases and hype.

---

## 21.3 Premature Optimization

### Intent (of the Anti‑Pattern)
*Attempting to optimise code before it is necessary, often at the expense of readability, maintainability, and correctness.*

### The Problem

Donald Knuth famously said: *“Premature optimization is the root of all evil.”* Developers often try to write “efficient” code from the start, using complex data structures, caching, or low‑level tricks. This leads to code that is harder to understand, more bug‑prone, and often no faster because the real bottlenecks are elsewhere.

#### Example: Premature Optimization

```typescript
// ❌ PREMATURE OPTIMIZATION: Overly complex from the start
class PrimeFinder {
  // Using a custom bit array to save memory (premature)
  private bits: Uint8Array;

  constructor(limit: number) {
    this.bits = new Uint8Array(Math.ceil(limit / 8));
    this.computePrimes(limit);
  }

  private computePrimes(limit: number): void {
    // Complex bit‑twiddling logic to mark composites
    for (let i = 2; i * i <= limit; i++) {
      if ((this.bits[Math.floor(i / 8)] & (1 << (i % 8))) === 0) {
        for (let j = i * i; j <= limit; j += i) {
          this.bits[Math.floor(j / 8)] |= (1 << (j % 8));
        }
      }
    }
  }

  isPrime(n: number): boolean {
    return (this.bits[Math.floor(n / 8)] & (1 << (n % 8))) === 0;
  }
}
```

**Problems**:
- The code is hard to read and maintain.
- For a typical application, a simple boolean array would be fast enough and much clearer.
- The optimisation may not even be needed (premature).

**Better**: Start with a simple, clear implementation. Measure performance. If it's too slow, profile to find the real bottleneck, then optimise that part.

```typescript
// ✅ SIMPLE FIRST
class PrimeFinder {
  private primes: boolean[];

  constructor(limit: number) {
    this.primes = new Array(limit + 1).fill(true);
    this.computePrimes(limit);
  }

  private computePrimes(limit: number): void {
    for (let i = 2; i * i <= limit; i++) {
      if (this.primes[i]) {
        for (let j = i * i; j <= limit; j += i) {
          this.primes[j] = false;
        }
      }
    }
  }

  isPrime(n: number): boolean {
    return this.primes[n];
  }
}
```

### When Optimisation is Not Premature

- You have measured and identified a bottleneck.
- You are writing a library that will be used in performance‑critical contexts.
- You have domain knowledge that the simple solution will definitely be too slow.

But in most cases, **correctness and clarity come first**.

---

## 21.4 Copy‑Paste Programming

### Intent (of the Anti‑Pattern)
*Reusing code by copying and pasting it, leading to duplication and maintenance nightmares.*

### The Problem

Copy‑paste programming is one of the most common anti‑patterns. A developer finds a piece of code that does something similar, copies it, and makes minor modifications. The result is multiple copies of essentially the same logic, each with slight variations. When a bug is found or a change is needed, you have to find and update every copy.

#### Example: Copy‑Paste in Validation

```typescript
// ❌ COPY-PASTE: Duplicated validation logic

// In user registration
function validateUser(email: string, password: string): boolean {
  if (!email.includes('@')) return false;
  if (password.length < 8) return false;
  return true;
}

// In profile update (copied and slightly modified)
function validateProfile(email: string, phone: string): boolean {
  if (!email.includes('@')) return false; // same email check
  if (phone && phone.length < 10) return false; // different phone check
  return true;
}

// In password reset (another copy)
function validatePasswordReset(email: string, newPassword: string): boolean {
  if (!email.includes('@')) return false;
  if (newPassword.length < 8) return false; // same password check as registration
  return true;
}
```

**Problems**:
- If the email validation rule changes (e.g., need to check for valid domains), you must update three places.
- Inconsistent behaviour may creep in if copies diverge.
- Code bloat.

**Solution**: Extract common logic into reusable functions or classes.

```typescript
// ✅ REUSABLE: Single source of truth
class Validator {
  static isValidEmail(email: string): boolean {
    return email.includes('@'); // could be more sophisticated
  }

  static isValidPassword(password: string): boolean {
    return password.length >= 8;
  }

  static isValidPhone(phone: string): boolean {
    return phone.length >= 10;
  }
}

// Now each use case calls the common validators
function validateUser(email: string, password: string): boolean {
  return Validator.isValidEmail(email) && Validator.isValidPassword(password);
}

function validateProfile(email: string, phone: string): boolean {
  return Validator.isValidEmail(email) && (phone ? Validator.isValidPhone(phone) : true);
}

function validatePasswordReset(email: string, newPassword: string): boolean {
  return Validator.isValidEmail(email) && Validator.isValidPassword(newPassword);
}
```

### Copy‑Paste in Larger Contexts

Copy‑paste often occurs at the service or component level too. For example, similar CRUD operations for different entities are copied and pasted. This can be addressed by generics, base classes, or code generation.

### The Right Way to Reuse Code

- **Extract common logic** into functions, classes, or modules.
- **Use inheritance or composition** to share behaviour.
- **Consider design patterns** like Template Method or Strategy when appropriate.
- **Write utility functions** for cross‑cutting concerns.

---

## Chapter Summary

Anti‑patterns are the dark side of design—they teach us what *not* to do. This chapter covered four common ones:

1. **Spaghetti Code and Ravioli Code** – Two extremes of poor structure. Spaghetti lacks organisation; ravioli is over‑fragmented. Aim for balanced, cohesive modules with clear responsibilities.

2. **God Object and Golden Hammer** – A class that does too much, and a favourite tool applied everywhere. Decompose large classes and choose patterns based on the problem, not habit.

3. **Premature Optimization** – Optimising before understanding the real bottlenecks leads to complex, hard‑to‑maintain code. Write clear code first, measure, then optimise where needed.

4. **Copy‑Paste Programming** – Duplication is the enemy of maintainability. Always extract shared logic into reusable components.

**Key Insight**: Recognising anti‑patterns is the first step toward avoiding them. When you spot these smells in your code, you have an opportunity to refactor towards cleaner, more sustainable designs.

---

## Next Chapter Preview

**Chapter 22: Refactoring to Patterns (Code Smells as Drivers, Step‑by‑Step Refactoring, TDD with Patterns)**

Knowing patterns and anti‑patterns is valuable, but the real skill lies in transforming messy code into well‑structured code. Chapter 22 explores how to use **code smells** to identify opportunities for applying patterns, how to refactor step‑by‑step without breaking the system, and how **Test‑Driven Development (TDD)** can guide you toward pattern‑based solutions.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../6. security_safety_critical_patterns/20. domain_driven_design.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='22. refactoring_to_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
